In [18]:
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from datetime import date

# 5. Создаем новые признаки

На диаграммах рассеивания видно, что графики некоторых переменных похожи на графики корня. Поэтому создадим такие признаки. \
У золота не очень похоже, но что-то такое есть, поэтому тоже добавим его сюда.

In [19]:
df = pd.read_csv('data.csv')
df['Date'] = pd.to_datetime(df['Date'])

df['NASDAQ_sqrt'] = np.sqrt(df['NASDAQ_price'])
df['S&P500_sqrt'] = np.sqrt(df['S&P500_price'])
df['GOLD_sqrt'] = np.sqrt(df['GOLD_price'])

Еще есть такое предположение, что по выходным активность на рынке меньше, из-за этого волатильность ниже, так что добавим категориальную переменную, которая показывает, является ли день рабочим (work day) или праздником/выходным (day off). Сначала напишем функцию, чтобы удобнее было.

In [20]:
class FinancialInfo:
    def __init__(self, start_date=None, end_date=None):
        # задание дефолтного значения первого дня
        if start_date is None:
            self.start_date = date(2010, 1, 1)
            self.start_date_text = self.start_date
        # задание пользовательского значения первого дня
        else:
            self.start_date_text = start_date
            sd = list(map(int, start_date.split('.')))
            self.start_date = date(sd[2], sd[1], sd[0])

        # задание дефолтного значения последнего дня
        if end_date is None:
            self.end_date = date.today()
            self.end_date_text = self.end_date
        # задание пользовательского значения последнего дня
        else:
            self.end_date_text = end_date
            ed = list(map(int, end_date.split('.')))
            self.end_date = date(ed[2], ed[1], ed[0])

    def get_holidays(self):
        '''Выгружает выходные и праздничные дни в соответствии с производственным календарем за даты, указанные в атрибутах объекта'''
        all_holidays = dict()

        years = np.arange(self.start_date.year, self.end_date.year + 1)
        months = np.arange(12)

        for y in years:
            calendar_url = f'https://calendar.yoip.ru/work/{y}-proizvodstvennyj-calendar.html'
            cal = requests.get(calendar_url,
                               headers={'User-Agent': UserAgent().chrome},
                               timeout=5)
            tree_cal = BeautifulSoup(cal.content, 'html.parser')
            for m in months:
                netraboty_m = [
                    int(i.text)
                    for i in tree_cal.find_all('table')[m].find_all(
                        'td', {'class': '_hd danger tt-hd'})
                ]
                netraboty_m.extend([
                    int(i.text)
                    for i in tree_cal.find_all('table')[m].find_all(
                        'td', {'class': '_hd warning tt-hd'})
                ])
                netraboty_m.extend([
                    int(i.text)
                    for i in tree_cal.find_all('table')[m].find_all(
                        'td', {'class': '_hd warning'})
                ])
                netraboty_m = [date(y, m + 1, i) for i in sorted(netraboty_m)]
                for day in netraboty_m:
                    all_holidays[day] = 'day off'

        calendar_df = pd.DataFrame.from_dict(all_holidays,
                                             orient='index',
                                             columns=['workday']).reset_index()
        calendar_df.rename(columns = {'index': 'Date'}, inplace = True)
        calendar_df['Date'] = pd.to_datetime(calendar_df['Date'])
        return calendar_df

    def get_all(self):
        # Создаем DataFrame с датами от start_date до end_date
        date_range = pd.date_range(start=self.start_date, end=self.end_date)
        df = pd.DataFrame(date_range, columns = ['Date'])
        df['Date'] = pd.to_datetime(df['Date'])

        df = pd.merge(df,
                      self.get_holidays(), on = 'Date',
                      how='left')

        df['workday'] = df['workday'].fillna('workday')

        return df

Применяем функцию, чтобы получить отдельную табличку с рабочими/выходными днями

In [21]:
days = FinancialInfo(start_date='01.01.2017', end_date='01.04.2024').get_all()
print(days.head())

        Date  workday
0 2017-01-01  day off
1 2017-01-02  day off
2 2017-01-03  day off
3 2017-01-04  day off
4 2017-01-05  day off


Мерджим датасет с этой табличкой и получаем готовый датасет с новым признаком!

In [22]:
df = pd.merge(df, days, on ='Date', how = 'left')
df.to_csv('data_new.csv')
df.head()

,Unnamed: 0,Date,NASDAQ_price,S&P500_price,WTI_price,GOLD_price,US Inflation,US Interest Rate,US Unemployment Rate,BTC_price,US President,NASDAQ_sqrt,S&P500_sqrt,GOLD_sqrt,workday
0,0,2024-04-01,16397.05,5257.97,83.14,2235.7,3.091,5.5,3.9,71333.48,Biden,128.050966,72.511861,47.283189,workday
1,1,2024-03-28,16377.23,5248.03,81.71,2193.6,3.091,5.5,3.8,69452.77,Biden,127.973552,72.443288,46.835884,workday
2,2,2024-03-27,16424.76,5226.31,81.29,2179.4,3.091,5.5,3.8,69991.90,Biden,128.159120,72.293222,46.684044,workday
3,3,2024-03-26,16446.85,5228.85,81.94,2182.7,3.091,5.5,3.8,69931.33,Biden,128.245273,72.310788,46.719375,workday
4,4,2024-03-25,16335.30,5219.52,80.85,2166.2,3.091,5.5,3.8,67234.09,Biden,127.809624,72.246246,46.542454,workday
